In [1]:
import pandas as pd
import numpy as np
import os
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Bidirectional
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
# dir_path = './' #local
dir_path = './drive/MyDrive/BAKA/' #colab

In [2]:
#uncomment in colab
from google.colab import drive
drive.mount('/content/drive')
# pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', sep=' ', index_col=False )
# test = np.genfromtxt(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', delimiter=' ',dtype='float64')
df = pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', header=None, sep=' ')
# df.drop(df.index[60:])
# filler = np.zeros((100-len(df), 31))
# df = df.append(pd.DataFrame(filler), ignore_index=True )
# df
if len(df.columns) > 31:
  df = df.drop(columns=[31])
# df = df.drop(df.index[60:])
# df
# sp_df = np.array_split(df, 2)
# sp_df[0]

Mounted at /content/drive


In [3]:
# for i in ['6']:
#     samples = os.listdir(dir_path + 'DataCollection' + '/' + i)
#     num_tests = int(len(samples)/5);
#     shuffle(samples, random_state = 0)
#     for k in range(0, len(samples)):
# #         df = np.genfromtxt(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], delimiter=' ',dtype='float64')
#         df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
#         df = df.drop(df.index[60:])
#         if len(df.columns) > 31:
#           df = df.drop(columns=[31])
#         if df.isnull().values.any():
#           print(i, " ", samples[k])
#         print(samples[k])
#         display(df)
#         # # if len(df) < 100:
#         # #     filler = np.zeros((100-len(df), 31))
#         # #     df = df.append(pd.DataFrame(filler), ignore_index=True )
#         # x_test.append(df.to_numpy())
#         # y_test.append(int(i));

In [4]:
files = os.listdir(dir_path + 'DataCollection')
files

['4', '7', '8', '0', '5', '2', '3', '6', '1', '9']

In [5]:
x_train = [];
y_train = [];

x_test = [];
y_test = [];
for i in files:
    samples = os.listdir(dir_path + 'DataCollection' + '/' + i)
    num_tests = int(len(samples)/5);
    shuffle(samples, random_state = 0)
    for k in range(0, num_tests):
#         df = np.genfromtxt(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], delimiter=' ',dtype='float64')
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        # df = df.drop(df.index[60:])
        if len(df.columns) > 31:
          df = df.drop(columns=[31])
        if df.isnull().values.any():
          print(i, " ", samples[k])
        # if len(df) < 100:
        #     filler = np.zeros((100-len(df), 31))
        #     df = df.append(pd.DataFrame(filler), ignore_index=True )
        x_test.append(df.to_numpy())
        y_test.append(int(i));
    
    for k in range(num_tests, len(samples)):
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        # df = df.drop(df.index[60:])
        if len(df.columns) > 31:
          df = df.drop(columns=[31])
        if df.isnull().values.any():
          print(i, " ", samples[k])
        # df = df.drop(df.index[100:])
        # if len(df) < 100:
        #     filler = np.zeros((100-len(df), 31))
        #     df = df.append(pd.DataFrame(filler), ignore_index=True )
        x_train.append(df.to_numpy())
        y_train.append(int(i));
    
    print(len(samples), ' ', num_tests, ' ', len(samples)- num_tests)
# print(len(x_train))
x_train = np.array(x_train)
y_train = np.array(y_train);

x_test = np.array(x_test)
y_test = np.array(y_test)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)


204   40   164
264   52   212
233   46   187
253   50   203
254   50   204
320   64   256
285   57   228
266   53   213
319   63   256
259   51   208
x_train.shape:  (2131, 90, 31)
y_train.shiape:  (2131,)
x_test.shape:  (526, 90, 31)
y_test.shape:  (526,)


In [6]:
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)



x_train.shape:  (2131, 90, 31)
y_train.shiape:  (2131,)
x_test.shape:  (526, 90, 31)
y_test.shape:  (526,)


In [7]:
def scale_data(data, min_max_scaler):
    for i in range(len(data)):
        data[i] = min_max_scaler.transform(data[i])
    return data

In [8]:
min_max_scaler = MinMaxScaler(feature_range=(0,1))

num_instances, num_time_steps, num_features = x_train.shape
x_train = np.reshape(x_train, newshape=(-1, num_features))
x_train = min_max_scaler.fit_transform(x_train)
x_train = np.reshape(x_train, newshape=(num_instances, num_time_steps, num_features))

x_train, y_train = shuffle(x_train, y_train, random_state=0)

num_instances, num_time_steps, num_features = x_test.shape
x_test = np.reshape(x_test, newshape=(-1, num_features))
x_test = min_max_scaler.transform(x_test)
x_test = np.reshape(x_test, newshape=(num_instances, num_time_steps, num_features))

x_test, y_test = shuffle(x_test, y_test, random_state=0)


In [9]:
# x_train = np.delete(x_train, [0,1,2,3,4,5,6,7,8], 0)
# y_train = np.delete(y_train, [0,1,2,3,4,5,6,7,8], 0)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)
# print(y_train)
# print("\n")
# print(y_test)

# y_train = y_train.astype('int')

# print(y_train)


x_train.shape:  (2131, 90, 31)
y_train.shiape:  (2131,)
x_test.shape:  (526, 90, 31)
y_test.shape:  (526,)


In [10]:
model = Sequential()
model.add(Bidirectional(LSTM(units=90, return_sequences=True ,dtype='float64'),input_shape=x_train.shape[1:],dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.6))

model.add(Bidirectional(LSTM(units=90 ,dtype='float64') ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.6))


# model.add(Dense(30, activation='softmax'))

model.add(Dense(len(files), activation='softmax',dtype='float64'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 90, 180)           87840     
_________________________________________________________________
batch_normalization (BatchNo (None, 90, 180)           720       
_________________________________________________________________
dropout (Dropout)            (None, 90, 180)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 180)               195120    
_________________________________________________________________
batch_normalization_1 (Batch (None, 180)               720       
_________________________________________________________________
dropout_1 (Dropout)          (None, 180)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [11]:
# model = Sequential()
# model.add(LSTM(units=60, input_shape=x_train.shape[1:], return_sequences=True,dtype='float64'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(LSTM(60, return_sequences=True ,dtype='float64'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(LSTM(60, return_sequences=True ,dtype='float64'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(LSTM(60,dtype='float64'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# # model.add(Dense(30, activation='softmax'))

# model.add(Dense(7, activation='softmax',dtype='float64'))
# model.summary()

In [12]:
opt = tf.keras.optimizers.Adam(lr=0.0001, decay=1e-5)

checkpoint_filepath = dir_path + 'Checkpoints/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)


gestures = model.fit(x = x_train,
            y = y_train,
            epochs=200,
            validation_split=0.1, #split 10% of the trainning set for the validation set,
            batch_size=24,
            callbacks=[model_checkpoint_callback],
            shuffle=True
         )

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/200
80/80 [==============================] - 14s 62ms/step - loss: 2.8237 - accuracy: 0.2514 - val_loss: 2.1176 - val_accuracy: 0.2944
Epoch 2/200
80/80 [==============================] - 3s 42ms/step - loss: 1.5352 - accuracy: 0.5149 - val_loss: 1.6264 - val_accuracy: 0.5701
Epoch 3/200
80/80 [==============================] - 3s 42ms/step - loss: 0.9965 - accuracy: 0.6667 - val_loss: 0.9356 - val_accuracy: 0.8271
Epoch 4/200
80/80 [==============================] - 3s 42ms/step - loss: 0.7138 - accuracy: 0.7621 - val_loss: 0.4677 - val_accuracy: 0.9065
Epoch 5/200
80/80 [==============================] - 3s 42ms/step - loss: 0.5529 - accuracy: 0.8211 - val_loss: 0.2229 - val_accuracy: 0.9626
Epoch 6/200
80/80 [==============================] - 3s 42ms/step - loss: 0.4240 - accuracy: 0.8581 - val_loss: 0.0902 - val_accuracy: 0.9766
Epoch 7/200
80/80 [==============================] - 3s 42ms/step - loss: 0.3468 - accuracy: 0.8925 - val_loss: 0.0538 - val_accuracy: 0.9860
Epoch

In [13]:
# model.save(dir_path + 'Models/gestures.h5')

In [14]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=24)
print("test loss, test acc:", results)
model.save(dir_path + 'Models/gestures_bidir', save_format='tf')

Evaluate on test data
22/22 [==============================] - 0s 15ms/step - loss: 0.6125 - accuracy: 0.9278
test loss, test acc: [0.6125446557998657, 0.927756667137146]


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_bidir/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_bidir/assets


In [15]:
# new_model = load_model(dir_path + 'Models/gestures.h5')
# print("Evaluate on test data")
# results = new_model.evaluate(x_test, y_test, batch_size=80)
# print("test loss, test acc:", results)

In [16]:
min_max_scaler.data_min_

array([   0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    , -591.101 ,  -12.7777,
       -190.142 , -626.233 ,    0.    , -238.535 , -648.629 ,    0.    ,
       -251.495 , -643.156 ,    0.    , -243.005 , -675.481 ,    0.    ,
       -220.466 , -607.075 ,    0.    , -158.459 ,    0.    ,    0.    ,
          0.    ])

In [18]:
min_max_scaler.data_max_

array([180.   , 180.   , 180.   , 179.999, 179.991, 180.   , 179.983,
       180.   , 179.985, 179.985, 539.053, 468.363, 720.07 , 581.654,
       522.811, 709.384, 585.01 , 508.522, 705.543, 567.779, 478.013,
       702.861, 544.45 , 478.722, 693.173, 499.893, 466.308, 707.228,
       127.401, 161.082, 127.977])

In [17]:
model.input

<KerasTensor: shape=(None, 90, 31) dtype=float64 (created by layer 'bidirectional_input')>